# Collecting Agent Logs for Monitoring

We have the working application so let's now start collecting logs.

First we will do this manually but later we will see how to do it with special software.

## Log Storage Strategy

We can imagine that these logs are saved to kafka or some logs collection system (logstash, fluentd, etc).

But here for simplicity we will put them in a file system.

## Creating Log Entries

Create `agent_logging.py`

Let's capture all the important information and create `create_log_entry`.

This function extracts all the essential information: agent configuration, tool usage, conversation history, token usage, and final output.

In [ ]:
from typing import List

import pydantic

from pydantic_ai import Agent
from pydantic_ai.usage import RunUsage
from pydantic_ai.messages import ModelMessage

from pydantic_ai.messages import ModelMessagesTypeAdapter


UsageTypeAdapter = pydantic.TypeAdapter(RunUsage)


def create_log_entry(
    agent: Agent,
    messages: List[ModelMessage],
    usage: RunUsage,
    output: str
):
    tools = []

    for ts in agent.toolsets:
        tools.extend(ts.tools.keys())

    dict_messages = ModelMessagesTypeAdapter.dump_python(messages)
    dict_usage = UsageTypeAdapter.dump_python(usage)

    return {
        "agent_name": agent.name,
        "system_prompt": agent._instructions,
        "provider": agent.model.system,
        "model": agent.model.model_name,
        "tools": tools,
        "messages": dict_messages,
        "usage": dict_usage,
        "output": output,
    }


It creates a log entry - a dictionary that later can easilty be serialized into JSON.

For conversions we use type adapters from Pydantic - to convert dataclasses to dictionaries.

## Handling Different Run Types

We need separate functions for streamed and regular runs:

In [ ]:
from pydantic_ai.run import AgentRunResult
from pydantic_ai.result import StreamedRunResult

async def log_streamed_run(
    agent: Agent,
    result: StreamedRunResult
):
    output = await result.get_output()
    usage = result.usage()
    messages = result.all_messages()

    log = create_log_entry(
        agent=agent,
        messages=messages,
        usage=usage,
        output=output
    )

    return log


def log_run(
    agent: Agent,
    result: AgentRunResult
):
    output = result.output
    usage = result.usage()
    messages = result.all_messages()

    log = create_log_entry(
        agent=agent,
        messages=messages,
        usage=usage,
        output=output
    )

    return log


The key difference is that streamed runs require awaiting the final output, while regular runs have it immediately available.

## Saving Logs to Filesystem

Finally, we save it to a JSON file.

For that, we first need custom serialization for complex objects:

In [ ]:
from datetime import datetime
from pydantic import BaseModel

def serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    if isinstance(obj, BaseModel):
        return obj.model_dump()
    raise TypeError(f"Type {type(obj)} not serializable")


And a helper function to extract the timestamp from the logs:

In [ ]:
def find_last_timestamp(messages):
    for msg in reversed(messages):
        if 'timestamp' in msg:
            return msg['timestamp']

Now save it:

In [ ]:
import json
import secrets
from pathlib import Path

def save_log(entry: dict):
    logs_folder = Path('logs')
    logs_folder.mkdir(exist_ok=True)

    ts = find_last_timestamp(entry['messages'])
    ts_str = ts.strftime("%Y%m%d_%H%M%S")
    rand_hex = secrets.token_hex(3)

    agent_name = entry['agent_name'].replace(" ", "_").lower()

    filename = f"{agent_name}_{ts_str}_{rand_hex}.json"
    filepath = logs_folder / filename

    with filepath.open("w", encoding="utf-8") as f_out:
        json.dump(entry, f_out, indent=2, default=serializer)

    return filepath


This creates unique filenames combining agent name, timestamp, and random hex to prevent conflicts.

## Integrating with Your Application

Include it in your app:

In [ ]:
log_entry = await log_streamed_run(agent, result)
log_file = save_log(log_entry)
print(f"\n\nLog saved to: {log_file}")

Now every agent interaction is automatically logged with complete context for monitoring and analysis.